# [1731. The Number of Employees Which Report to Each Employee](https://leetcode.com/problems/the-number-of-employees-which-report-to-each-employee/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Employees

<pre>+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| reports_to  | int      |
| age         | int      |
+-------------+----------+</pre>
employee_id is the column with unique values for this table.
This table contains information about the employees and the id of the manager they report to. Some employees do not report to anyone (reports_to is null). 
 

For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.

 

Example 1:

Input: 
Employees table:
<pre>+-------------+---------+------------+-----+
| employee_id | name    | reports_to | age |
+-------------+---------+------------+-----+
| 9           | Hercy   | null       | 43  |
| 6           | Alice   | 9          | 41  |
| 4           | Bob     | 9          | 36  |
| 2           | Winston | null       | 37  |
+-------------+---------+------------+-----+</pre>
Output: 
<pre>+-------------+-------+---------------+-------------+
| employee_id | name  | reports_count | average_age |
+-------------+-------+---------------+-------------+
| 9           | Hercy | 2             | 39          |
+-------------+-------+---------------+-------------+</pre>
Explanation: Hercy has 2 people report directly to him, Alice and Bob. Their average age is (41+36)/2 = 38.5, which is 39 after rounding it to the nearest integer.

In [1]:
# Pandas schema

import pandas as pd

data = [[9, 'Hercy', None, 43], [6, 'Alice', 9, 41], [4, 'Bob', 9, 36], [2, 'Winston', None, 37]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'reports_to', 'age']) \
    .astype({'employee_id': 'Int64', 'name': 'object', 'reports_to': 'Int64', 'age': 'Int64'})

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# To pyspark schema

spark = SparkSession.builder.getOrCreate()

schema = StructType([
    StructField("employee_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("reports_to", IntegerType(), True),
    StructField("age", IntegerType(), True)
])

employees_df = spark.createDataFrame(data, schema=schema)
employees_df.show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 12:20:44 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 12:20:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 12:20:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+-------+----------+---+
|employee_id|   name|reports_to|age|
+-----------+-------+----------+---+
|          9|  Hercy|      null| 43|
|          6|  Alice|         9| 41|
|          4|    Bob|         9| 36|
|          2|Winston|      null| 37|
+-----------+-------+----------+---+


In [3]:
# In spark dataframe
from pyspark.sql import functions as F

employees_df \
    .filter(~F.isnull('reports_to')).groupBy('reports_to') \
    .agg(F.round(F.avg('age'), 0).alias('average_age'), F.count('*').alias('reports_count')) \
    .withColumn('average_age', F.col('average_age').cast('int')) \
    .alias('a') \
    .join(employees_df, F.col('a.reports_to') == employees_df['employee_id'], how='inner') \
    .select('employee_id', 'name', 'reports_count', 'average_age') \
    .orderBy('employee_id')\
    .show()

+-----------+-----+-------------+-----------+
|employee_id| name|reports_count|average_age|
+-----------+-----+-------------+-----------+
|          9|Hercy|            2|         39|
+-----------+-----+-------------+-----------+


In [4]:
# In spark SQL

employees_df.createOrReplaceTempView('employees')

spark.sql('''
select e1.reports_to as employee_id, e2.name, count(e1.reports_to) as reports_count, cast(round(avg(e1.age),0) as int) as average_age
from employees e1
left join (select name, employee_id from employees) e2
on e2.employee_id = e1.reports_to
where e1.reports_to is not null
group by e1.reports_to, e2.name
order by employee_id;
''').show()

+-----------+-----+-------------+-----------+
|employee_id| name|reports_count|average_age|
+-----------+-----+-------------+-----------+
|          9|Hercy|            2|         39|
+-----------+-----+-------------+-----------+


In [5]:
spark.stop()